In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dset
from torch.utils.data import DataLoader
import torch.optim as optim 
from tqdm import tqdm
import torch.nn as nn

import PracResNet

In [2]:
learning_rate = 0.001
num_epoch = 10
batch_size = 200

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = dset.ImageNet('../../../media/data1/data/Imagenet', split='train', transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

testset = dset.ImageNet('../../../media/data1/data/Imagenet', split='val', transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

In [4]:
device = torch.device("cuda:3" if torch.cuda.is_available() else None)
print('Now Device: ', device)

Now Device:  cuda:3


In [5]:
model = PracResNet.ResNet(PracResNet.Bottleneck, [3,4,6,3],).to(device)
print('My model:\n','#'*100,"\n",model)

My model:
 #################################################################################################### 
 ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu

In [6]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
loss_arr = []

for i in range(num_epoch): # epoch만큼 학습 반복
    train_bar = tqdm(trainloader)
    running_results = {'batch_sizes':0, 'loss':0}
    model.train()
    for j,[image,label] in enumerate(train_bar): # train set으로 학습하기
        running_results['batch_sizes'] += batch_size
        x = image.to(device) # 입력 데이터를 GPU에 적재
        y_= label.to(device) # 라벨 데이터를 GPU에 적재
        
        optimizer.zero_grad() # 학습마다 기울기를 0으로 초기화
        output = model.forward(x) # 순전파한 후, 결과를 저장
        loss = loss_func(output,y_) # 예측값과 결과값의 loss를 구함
        loss.backward() # 가중치와 편향을 계산
        optimizer.step() # 가중치 업데이트
        
        running_results['loss'] += loss.item() * batch_size
        
        train_bar.set_description(desc = '[%d/%d] Loss: %.4f' % (
            i, num_epoch, running_results['loss'] / running_results['batch_sizes'],
        ))
#         if j % 1000 == 0:
#             print(i, ":", loss)
#             loss_arr.append(loss.cpu().detach().numpy())

[0/10] Loss: 7.4784:   0%|▌                                                                                                                                               | 24/6406 [01:10<4:59:25,  2.81s/it]